In [1]:
import os
import pandas as pd


folder_path = 'death_rates/Mx_1x1/'
file_names = os.listdir(folder_path)

all_data = []

# Loop through all files in the folder
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    
    with open(file_path, 'r') as file:
        # get country name in the first line
        country = file.readline().strip().split(',')[0]
        
    # skip first 3 rows to remove the unuseful information
    df = pd.read_csv(file_path, delim_whitespace=True, skiprows=3, 
                     names=['Year', 'Age', 'Female', 'Male', 'Total'])
    # Convert the string of the total column to a number
    df['Total'] = pd.to_numeric(df['Total'], errors='coerce')
    df.dropna(subset=['Total'], inplace=True)
    
    # get average death rate for each year
    avg_death_rate = df.groupby('Year')['Total'].mean().reset_index()
    # add country column
    avg_death_rate['Country'] = country
    # final file header
    avg_death_rate.columns = ['Year', 'Death_Rate', 'Country']
    all_data.extend(avg_death_rate.to_dict('records'))

# to DataFrame
data = pd.DataFrame(all_data)
# sort by year
data.sort_values(by=['Year'], inplace=True)
# save to csv
data.to_csv('clean_data.csv', index=False)